In [1]:
# # Run this and then restart the kernel at the start of each session to install
# # 'teotil3' in development mode
# !pip install -e /home/jovyan/projects/teotil3/

In [2]:
import os
import pprint
import shutil

import nivapy3 as nivapy
import pandas as pd
import teotil3 as teo
import utils
import yaml

In [3]:
eng = nivapy.da.connect_postgis()

Connection successful.


# OsloMod scenarios

## 1. Scenarios overview

### 1.1. Baseline

The `Baseline` scenario is defined as:

 * Agricultural measures from 2021 assumed to be in place for all years during the simulation period (2017 to 2019). Measures are those registered in RMP in 2021.

 * For "large" wastewater plants, use "real" data for 2017 to 2019 i.e. measured data where available, and otherwise estimated/interpolated values based on the number of people and fritidsboliger connected to each plant.

 * Assume 2% overflow for all wastewater sites larger than 1000 p.e. For example, if a site has a measured inflow of 1000 tonnes of TOTP and a measured outflow of 400 tonnes, assume nutrient losses at the outflow co-ordinates of 400 + 0.02*1000 = 420 tonnes.

### 1.2. Scenario A ("Vedtatte tiltak – effekt 2033")

In `Scenario A`:

 * Agricultural measures are added to the five "miljøkravsoner", corresponding to "forskrifter om miljøkrav" (described at https://lovdata.no/). Outside of the miljøkravsoner, baseline conditions are maintained.

 * All WWTPs larger than 10 000 p.e. are upgraded to `Kjemisk-biologisk m/N-fjerning`, with minimum treatment efficiencies of 80% for TOTN, 95% for BOF5, 95% for KOF and 95% for SS. Any sites that already have better treatment efficiencies than this are not changed (e.g. if a site already has 85% efficiency for TOTN in the baseline, it will keep this efficiency in the scenario).

 * Assume 2% overflow from all sites larger than 1000 p.e. (the same as for the baseline).

### 1.3. Scenario B ("Ambisiøst scenarie – effekt ca 2040")

In `Scenario B`:

 * Agricultural measures are maximised everywhere - both inside and outside of the miljøkravsoner.

 * For WWTPs, all measures from Scenario A are included (i.e. the treatment type is changed to `Kjemisk-biologisk m/N-fjerning` and treatment efficiencies are at least 80% for TOTN, 95% for BOF5, 95% for KOF and 95% for SS for all sites >10 000 p.e.).

 * In addition, sites >10 000 p.e. with outflows **directly** to Oslofjorden are upgraded to have at least 85% efficiency for TOTN. The list of sites with direct discharges is shown below and is the same as those originally defined as "internal sources" in the Martini model. See e-mail from Phil received 08.09.2025 for details.

 * WWTPs with capacity between 5000 to 10 000 p.e. are upgraded to have at least 70% efficiency for TOTN. However, the site type ("renseprinsipp") is not changed.

 * Overflow is reduced to 1% of the inflow for sites >1000 p.e.

In [4]:
# Show sites with "direct discharges" according to Martini
direct_df = pd.read_csv(r"../data/of800_wwtp_direct_to_oslofjord.csv").query(
    "`max_capacity_2017-19` >= 10000"
)
direct_df

,kilderefnr,anlegg_nr,name,max_capacity_2017-19
0,0220AL01,3203.0059.01,Sentralrenseanlegg Vest (VEAS),926610
1,0301AL01,0301.0979.01,Bekkelaget renseanlegg med tilførselstuneller ...,746333
2,0704AL40,3905.0110.01,Tønsberg renseanlegg,181337
3,0602AL06,3301.0187.01,Solumstrand avløpsanlegg,150663
4,0106AL00,3107.0134.01,Øra avløpsanlegg,129451
5,0136AL00,3103.0153.01,Fuglevik avløpsanlegg,98306
6,0706AL15,3907.0070.01,Sandefjord renseanlegg,85687
7,0214AL23,3218.0016.01,Nordre Follo renseanlegg,81225
8,0626AL61,3312.0091.01,Linnes avløpsanlegg,57020
9,0709AL01,3909.0040.01,Lillevik renseanlegg,56333


### 1.4. Defining site capacity

Miljødirektoratet's database includes several capacity metrics for WWTPs, which do not always agree. The analysis here uses the following attributes in order of priority (based on advice from Torstein - see e-mail received 04.09.2025):

 1. `Tilført mengde til avløpsanlegget inkl. overløp, i pe`
    
 2. `Tilført mengde til avløpsanlegget inkl. overløp, i pe, beregnet av Forurensning ut fra BOF5 og fmaks = 1,5`

 3. `Dimensjonerende kapasitet, i pe`


### 1.5. Scenarios schema

The scenarios are defined in `scenarios.yaml`.

## 2. Model options 

In [5]:
# Scenario definitions
scen_yaml = "../data/scenarios.yaml"

# Period of interest
st_yr, end_yr = 2017, 2019
nve_year = 2024

# AGRICAT options
agri_loss_model = "annual"
agri_version = "20250912"

# Vassdragsområder of interest
vassom_list = range(1, 18)

# Data paths
teo3_base_dir = r"/home/jovyan/shared/common/teotil3"
oslomod_scen_dir = r"/home/jovyan/shared/common/oslofjord_modelling/phase3_scenarios"

# Whether to delete the scenario folder first if it already exists
delete_existing = True

In [6]:
# Read scenario definitions
with open(scen_yaml, "r") as file:
    scenarios_dict = yaml.safe_load(file)
pprint.pp(scenarios_dict)

{'Baseline': {'agri_scen_name': 'Baseline', 'overflow': {'1000-1e10': 2}},
 'Scenario_A': {'agri_scen_name': 'SCA',
                'overflow': {'1000-1e10': 2},
                'upgrade_by_capacity': {'10000-1e10': {'type': None,
                                                       'totn': 80,
                                                       'ss': 95,
                                                       'bof5': 95,
                                                       'kof': 95}}},
 'Scenario_B': {'agri_scen_name': 'SCB',
                'overflow': {'1000-1e10': 1},
                'upgrade_by_capacity': {'5000-10000': {'type': None,
                                                       'totn': 70},
                                        '10000-1e10': {'type': None,
                                                       'totn': 80,
                                                       'ss': 95,
                                                       'bof5': 95,
         

## 3. Generate model input files

In [7]:
# Get regines for area of interest
reg_gdf = utils.get_vassom_regines(vassom_list, eng, 2023)

for scen, scen_data in scenarios_dict.items():
    print("\n\n", scen)

    # Create scenario folder structure
    scen_data_fold = os.path.join(oslomod_scen_dir, "teotil3_input_files", scen)
    if os.path.exists(scen_data_fold) and delete_existing:
        shutil.rmtree(scen_data_fold)
    os.makedirs(scen_data_fold)

    for year in range(st_yr, end_yr + 1):
        # Create a temporary folder for modified annual data
        scen_ann_data_fold = os.path.join(scen_data_fold, str(year))
        os.makedirs(scen_ann_data_fold)

        # Read unmodified TEOTIL3 baseline file
        teo3_csv = os.path.join(
            teo3_base_dir,
            "annual_input_data",
            f"agri_{agri_loss_model}_loss",
            f"teotil3_input_data_nve{nve_year}_{year}.csv",
        )
        df = pd.read_csv(teo3_csv)

        # Filter to region of interest
        df["vassom"] = df["regine"].str.split(".", n=1, expand=True)[0].astype(int)
        df = df.query("vassom in @vassom_list").reset_index(drop=True)
        del df["vassom"]

        # Read agri scenario data
        agri_scen = scen_data.get("agri_scen_name")
        if agri_scen:
            agri_scen_dir = os.path.join(
                oslomod_scen_dir, "agri_raw", f"agri_data_v{agri_version}"
            )
            agri_df = utils.read_raw_agri_scen_data(
                year, agri_scen, agri_scen_dir, agri_loss_model
            )

            # Apply agri scenario
            df = utils.apply_agri_scenario(df, agri_df)

        # Read raw wastewater data
        ww_df = utils.read_raw_wastewater_data(year)

        # Upgrade plants based on capacity
        ww_df = utils.upgrade_sites_by_capacity(ww_df, scen_data)

        # Upgrade a specific list plants
        ww_df = utils.upgrade_sites_by_id(ww_df, scen_data)

        # Add overflows
        # NOTE: This step must be done AFTER other upgrades!
        ww_df = utils.estimate_overflows(ww_df, scen_data)

        # Save temporary file
        xl_path = os.path.join(scen_ann_data_fold, f"large_wastewater_{year}_raw.xlsx")
        ww_df.to_excel(xl_path, index=False)

        # Generate model input file
        utils.apply_wastewater_scenario(
            df,
            reg_gdf,
            scen_data_fold,
            year,
            eng,
            scen,
        )

        # Delete temp folder
        # shutil.rmtree(scen_ann_data_fold)



 Baseline
1 locations do not have outlet co-ordinates in this year's data.
         site_id                                      name
90  2311.0001.01  Hav Line - Slakteskipet Norwegian Gannet


 Scenario_A
1 locations do not have outlet co-ordinates in this year's data.
         site_id                                      name
90  2311.0001.01  Hav Line - Slakteskipet Norwegian Gannet


 Scenario_B
1 locations do not have outlet co-ordinates in this year's data.
         site_id                                      name
90  2311.0001.01  Hav Line - Slakteskipet Norwegian Gannet
